In [1]:
import pandas as pd
import numpy as np
import sdv
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

/opt/homebrew/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [ ]:
df = pd.read_csv("../data/data.csv")